In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/nursery/nursery.data

--2019-04-27 13:38:58--  https://archive.ics.uci.edu/ml/machine-learning-databases/nursery/nursery.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1059371 (1.0M) [application/x-httpd-php]
Saving to: ‘nursery.data.1’

nursery.data.1      100%[===================>]   1.01M  5.56MB/s    in 0.2s    

2019-04-27 13:38:58 (5.56 MB/s) - ‘nursery.data.1’ saved [1059371/1059371]



In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
import sys
np.set_printoptions(threshold=1000)
from sklearn.model_selection import train_test_split
import random
import operator
import math

In [3]:
df = pd.read_csv('nursery.data',header=None)
print(df.shape)
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore',sparse='False')
label = df.iloc[:,-1]
df2 = df.iloc[:,0:8]
df2 = enc.fit_transform(df2)
enc1 = OrdinalEncoder()
label1 = label.values.reshape(-1,1)
label1 = enc1.fit_transform(label1)
X = df2
y = label1
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
from scipy.sparse import csr_matrix
X_train = csr_matrix(X_train).toarray()
X_test = csr_matrix(X_test).toarray()
y_train = csr_matrix(y_train).toarray()
y_test = csr_matrix(y_test).toarray()
trainingSet = np.c_[X_train,y_train]
testSet = np.c_[X_test,y_test]
print(enc1.categories_)

(12960, 9)
[array(['not_recom', 'priority', 'recommend', 'spec_prior', 'very_recom'],
      dtype=object)]


In [0]:
def euclideanDistance(instance1, instance2, length):
	distance = 0
	for x in range(length):
		distance += pow((instance1[x] - instance2[x]), 2)
	return math.sqrt(distance)

def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append((trainingSet[x], dist))
	distances.sort(key=operator.itemgetter(1))
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])
	return neighbors

def getResponse(neighbors):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][-1]
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes[0][0]

def getAccuracy(testSet, predictions):
  correct = 0
  for x in range(len(testSet)):
#     print(testSet[x][-1] - predictions[x])
#     print(predictions[x])
    if testSet[x][-1] - predictions[x] == 0:
      correct += 1
  return (correct/float(len(testSet))) * 100.0

In [0]:
k = 2
i = 0
predictions = []
for x in range(len(testSet)):
  neighbors = getNeighbors(trainingSet, testSet[x], k)
  result = getResponse(neighbors)
  predictions.append(result)
  print(i)
  i = i+1
# 	print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))


In [0]:
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + str(accuracy) + '%')
# print(testSet[:-1])
# print(predictions)

Accuracy: 78.96090534979425%


In [0]:
np.unique(y_train)

array([0., 1., 2., 3., 4.])

In [0]:
t = testSet[:,-1]
p = predictions
from sklearn.metrics import confusion_matrix
print(confusion_matrix(t,p))
from sklearn.metrics import precision_score,recall_score
print("Macro Precision: "+ str(precision_score(t, p, average='macro')))
print("Micro Precision: "+ str(precision_score(t, p, average='micro')))
print("Weighted Precision: " + str(precision_score(t, p, average='weighted')))


print("Macro Recall: "+ str(recall_score(t, p, average='macro')))
print("Micro Recall: "+ str(recall_score(t, p, average='micro')))
print("Weighted Recall: " + str(recall_score(t, p, average='weighted')))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(t,p))

print("Precision: "+ str(precision_score(t, p, average=None)))
print("Recall: "+ str(recall_score(t, p, average=None)))

[[1173   85    0   61    6]
 [  70  936    0  208   64]
 [   0    0    0    0    1]
 [  61  221    0  908    0]
 [   5   36    0    0   53]]
Macro Precision: 0.565474092671798
Micro Precision: 0.7896090534979424
Weighted Precision: 0.7925781812482898
Macro Recall: 0.5889064764766367
Micro Recall: 0.7896090534979424
Weighted Recall: 0.7896090534979424
[[1173   85    0   61    6]
 [  70  936    0  208   64]
 [   0    0    0    0    1]
 [  61  221    0  908    0]
 [   5   36    0    0   53]]
Precision: [0.8961039  0.73239437 0.         0.77145285 0.42741935]
Recall: [0.88528302 0.73239437 0.         0.76302521 0.56382979]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=4)

#Train the model using the training sets
knn.fit(X_train, y_train.ravel())

#Predict the response for test dataset
y_pred = knn.predict(X_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100)


Accuracy: 92.05246913580247


In [0]:
print("Precision: "+ str(precision_score(y_test, y_pred, average=None)))
print("Recall: "+ str(recall_score(y_test, y_pred, average=None)))

Precision: [0.99699022 0.82814614 0.         0.9592233  0.82352941]
Recall: [1.         0.95774648 0.         0.8302521  0.44680851]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
